# Importing Libraries

In [1]:
import bokeh
import numpy as np 
import pandas as pd 
from datetime import datetime
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import HoverTool, ColumnDataSource, Range1d, FactorRange
from bokeh.layouts import gridplot,layout
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
output_notebook()

import warnings
warnings.filterwarnings('ignore')
#pd.options.display.max_rows = None
#pd.options.display.max_columns = None

Loading BokehJS ...

In [2]:
import bokeh
bokeh.__version__

'2.0.1'

# Load Data

In [3]:
description = pd.read_csv('Covid19Canada-master/codebook.csv')

n_cases = pd.read_csv('Covid19Canada-master/cases.csv')
n_death = pd.read_csv('Covid19Canada-master/mortality.csv')

n_recovered = pd.read_csv('Covid19Canada-master/recovered_cumulative.csv')
n_testing = pd.read_csv('Covid19Canada-master/testing_cumulative.csv')

color = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)] 

In [4]:
display(description)

,Variable,Description,Label
0,case_id,National Case ID Number,NaN
1,provincial_case_id,Provincial Case ID Number,NaN
2,age,"Age, if specific age not given then range prov...",NaN
3,sex,Sex,NaN
4,health_region,"Health region, if reported",NaN
5,province,Province,NaN
6,country,Country,NaN
7,date_report,Reported date (i.e. public announcement date) ...,NaN
8,report_week,Week of Report (Sundays are 1st day of week),NaN
9,travel_yn,Travel history (yes/no),"0=no, 1=yes, Not Reported"


# Introduction


I want to analysis the publicly available information on confirmed and presumptive postive cases during the ongoing COVID-19 outbreak in Canada with Bokeh for data visualization. The data comes from the github ishaberry, which they’ve shared publicly in https://github.com/ishaberry/Covid19Canada.



# Number of Cases in Canada

In [5]:
display(n_cases.head(1))
n_cases.shape

,case_id,provincial_case_id,age,sex,health_region,province,country,date_report,report_week,travel_yn,travel_history_country,locally_acquired,case_source,additional_info,additional_source
0,1,1,50-59,Male,Toronto,Ontario,Canada,25-01-2020,19-01-2020,1,China,NaN,(1) https://news.ontario.ca/mohltc/en/2020/01/...,NaN,NaN


(6320, 15)

### Setting 

In [6]:
n_cases = n_cases.iloc[:,:11]
n_cases['date_report'] = pd.to_datetime(n_cases['date_report'], dayfirst = True)
n_cases['report_week'] = pd.to_datetime(n_cases['report_week'], dayfirst = True)
n_cases['people'] = 1

In [7]:
all_gender = n_cases.groupby(['sex'])['people'].sum().sort_values(ascending = False).reset_index();
all_gender_factors = pd.unique(all_gender['sex'])
all_gender_data = ColumnDataSource(all_gender)

all_age = n_cases.groupby(['age'])['people'].sum().sort_values(ascending = False).reset_index();
all_age_factors = pd.unique(all_age['age'])
all_age_data = ColumnDataSource(all_age)

sex_case = n_cases.loc[n_cases['sex'] != 'Not Reported']
sex_case = sex_case.groupby(['sex'])['people'].sum().sort_values(ascending = False).reset_index();
sex_case_factors = pd.unique(sex_case['sex'])
sex_case_data = ColumnDataSource(sex_case)

age_case = n_cases.loc[n_cases['age'] != 'Not Reported']
age_case = age_case.groupby(['age'])['people'].sum().sort_values(ascending = False).reset_index();
age_case_factors = pd.unique(age_case['age'])
age_case_data = ColumnDataSource(age_case)

sex_age = n_cases.loc[(n_cases['age'] != 'Not Reported') & (n_cases['sex'] != 'Not Reported')]
sex_age = sex_age.groupby(['age', 'sex'])['people'].sum().reset_index()
sex_age['label'] = tuple(zip(sex_age['age'], sex_age['sex']))
sex_age_data = ColumnDataSource(sex_age)

### Confirmed Cases By Gender and Age

This chart shows the number of confirmed cases in gender and age.

In [8]:
plot1 = figure(title = "Confirmed Cases By Gender", plot_width = 495, plot_height = 300, x_range = FactorRange(factors = all_gender_factors),background_fill_color="#fafafa")
plot2 = figure(title = "Confirmed Cases By Age",plot_width = 495, plot_height = 300, x_range = FactorRange(factors = all_age_factors),background_fill_color="#fafafa")
plot3 = figure(title = "Confirmed Cases By Gender",plot_width = 495, plot_height = 300, x_range = FactorRange(factors = sex_case_factors),background_fill_color="#fafafa")
plot4 = figure(title = "Confirmed Cases By Age",plot_width = 495, plot_height = 300, x_range = FactorRange(factors = age_case_factors),background_fill_color="#fafafa")
plot5 = figure(title = "Confirmed Cases By Gender & Age",plot_width = 990, plot_height = 300, x_range = FactorRange(factors = sex_age['label']),background_fill_color="#fafafa")

plot1.vbar(x = 'sex' , top = 'people' , width = 0.5, source = all_gender_data)
plot2.vbar(x = 'age' , top = 'people' , width = 0.8, source = all_age_data)
plot3.vbar(x = 'sex' , top = 'people' , width = 0.3, source = sex_case_data)
plot4.vbar(x = 'age' , top = 'people' , width = 0.8, source = age_case_data)
plot5.vbar(x = 'label' , top = 'people' , width = 0.8, source = sex_age_data,\
          fill_color=factor_cmap('label', factors= sex_age['sex'] , start=1, end=2, palette = Spectral6))

plot1.add_tools(HoverTool(tooltips = [('Cases', '@people')]))
plot2.add_tools(HoverTool(tooltips = [('Cases', '@people')]))
plot3.add_tools(HoverTool(tooltips = [('Cases', '@people')]))
plot4.add_tools(HoverTool(tooltips = [('Cases', '@people')]))
plot5.add_tools(HoverTool(tooltips = [('Cases', '@people')]))

plot2.xaxis.major_label_orientation = 1
plot4.xaxis.major_label_orientation = 1
plot5.xaxis.major_label_orientation = 1

plot1.xaxis.axis_label = 'Gender'
plot1.yaxis.axis_label = 'Number of Confirmed Cases'
plot2.xaxis.axis_label = 'Age'
plot2.yaxis.axis_label = 'Number of Confirmed Cases'
plot3.xaxis.axis_label = 'Gender'
plot3.yaxis.axis_label = 'Number of Confirmed Cases'
plot4.xaxis.axis_label = 'Age'
plot4.yaxis.axis_label = 'Number of Confirmed Cases'
plot5.xaxis.axis_label = 'Gender & Age'
plot5.yaxis.axis_label = 'Number of Confirmed Cases'

grid = layout([[plot1,plot2],[plot3, plot4],[plot5]]) 

show(grid)

### Setting 

In [9]:
data_report = pd.DataFrame(n_cases['date_report'].value_counts()).reset_index().sort_values(by = 'index')
data_report_data = ColumnDataSource(data_report)
report_week = pd.DataFrame(n_cases['report_week'].value_counts()).reset_index().sort_values(by = 'index')
report_week_data = ColumnDataSource(report_week)

### Confirmed Cases Over Time
This chart shows the number of confirmed cases on each date.

In [10]:
plot1 = figure(title = "Confirmed Cases By Date",plot_width = 900, plot_height = 300,x_axis_type="datetime",background_fill_color="#fafafa")
plot2 = figure(title = "Confirmed Cases By Week",plot_width = 900, plot_height = 300,x_axis_type="datetime",background_fill_color="#fafafa")
plot3 = figure(title = "Confirmed Cases By Date & Week",plot_width = 900, plot_height = 300,x_axis_type="datetime",background_fill_color="#fafafa")

plot1.line(x = 'index' , y = 'date_report' , width = 2, source = data_report_data);
plot1.circle(x = 'index', y = 'date_report', size= 5, source = data_report_data);

plot2.line(x = 'index' , y = 'report_week' , width = 2, source = report_week_data);
plot2.circle(x = 'index', y = 'report_week', size= 5, source = report_week_data);

line1 = plot3.line(x = 'index' , y = 'date_report' , width = 2, source = data_report_data)
line2 = plot3.line(x = 'index' , y = 'report_week' , width = 2, source = report_week_data, color= 'orange')
circle1 = plot3.circle(x = 'index', y = 'date_report', size= 5, source = data_report_data)
circle2 = plot3.circle(x = 'index', y = 'report_week', size= 5, source = report_week_data,color= 'orange')

plot1.add_tools(HoverTool(tooltips = [('Cases', '@date_report')]))
plot2.add_tools(HoverTool(tooltips = [('Cases', '@report_week')]))
plot3.add_tools(HoverTool(renderers=[line1], tooltips = [('Cases', '@date_report')]))
plot3.add_tools(HoverTool(renderers=[line2], tooltips = [('Cases', '@report_week')]))

plot1.xaxis.axis_label = 'Date'
plot1.yaxis.axis_label = 'Number of Confirmed Cases'
plot2.xaxis.axis_label = 'Date'
plot2.yaxis.axis_label = 'Number of Confirmed Cases'
plot3.xaxis.axis_label = 'Date'
plot3.yaxis.axis_label = 'Number of Confirmed Cases'

grid = layout([[plot1],[plot2],[plot3]])

show(grid)

### Setting 

In [11]:
province = (pd.DataFrame(n_cases['province'].value_counts()).reset_index()
                 .sort_values(by = 'province',ascending = False))
province_factors = pd.unique(province['index'])
province_data = ColumnDataSource(province)

health_region = (pd.DataFrame(n_cases['health_region'].value_counts()).reset_index()
                 .sort_values(by = 'health_region',ascending = False))
health_region_factors = pd.unique(health_region['index'])
health_region_data = ColumnDataSource(health_region)

### Confirmed Cases  By Region
This chart shows the number of confirmed cases in each province and health region.

In [12]:
plot1 = figure(title = "Confirmed Cases By Province",plot_width = 900, plot_height = 300,x_range = FactorRange(factors = province_factors),background_fill_color="#fafafa")
plot2 = figure(title = "Confirmed Cases By Health Region",plot_width = 990, plot_height = 500,x_range = FactorRange(factors = health_region_factors),background_fill_color="#fafafa")

plot1.vbar(x = 'index' , top = 'province' , width = 0.2, source = province_data);
plot2.vbar(x = 'index' , top = 'health_region' , width = 0.5, source = health_region_data);
plot1.add_tools(HoverTool(tooltips = [('Cases', '@province')]))
plot2.add_tools(HoverTool(tooltips = [('Cases', '@health_region')]))

plot1.xaxis.major_label_orientation = 1
plot2.xaxis.major_label_orientation = 1

plot1.xaxis.axis_label = 'Province'
plot1.yaxis.axis_label = 'Number of Confirmed Cases'
plot2.xaxis.axis_label = 'Health Region'
plot2.yaxis.axis_label = 'Number of Confirmed Cases'

grid = layout([[plot1],[plot2]]) 

show(grid)

### Setting 

In [13]:
n_travel = n_cases.loc[n_cases['travel_yn'] != 'Not Reported']
n_travel = (pd.DataFrame(n_travel['travel_yn'].value_counts()).reset_index()
                 .sort_values(by = 'travel_yn',ascending = False))
n_travel_factors = pd.unique(n_travel['index'])
n_travel_data = ColumnDataSource(n_travel)

travel_history_country = (pd.DataFrame(n_cases['travel_history_country'].value_counts()).reset_index()
                 .sort_values(by = 'travel_history_country',ascending = False))
travel_history_country_factors = pd.unique(travel_history_country['index'])
travel_history_country_data = ColumnDataSource(travel_history_country)

### Number of Travel History

This chart shows the number of travel history and where are the travllers coming from.

In [14]:
plot1 = figure(title = "Travel Cases", plot_width = 200, plot_height = 500,x_range = FactorRange(factors = n_travel_factors),background_fill_color="#fafafa")
plot2 = figure(title = "Travel History",plot_width = 780, plot_height = 500,x_range = FactorRange(factors = travel_history_country_factors),background_fill_color="#fafafa")

plot1.vbar(x = 'index' , top = 'travel_yn' , width = 0.2, source = n_travel_data);
plot2.vbar(x = 'index' , top = 'travel_history_country' , width = 0.8, source = travel_history_country_data);
plot1.add_tools(HoverTool(tooltips = [('Cases', '@travel_yn')]))
plot2.add_tools(HoverTool(tooltips = [('Cases', '@travel_history_country')]))

plot2.xaxis.major_label_orientation = 1

plot1.xaxis.axis_label = 'Travel History'
plot1.yaxis.axis_label = 'Number of Confirmed Cases'
plot2.xaxis.axis_label = 'Travel Area'
plot2.yaxis.axis_label = 'Number of Confirmed Cases'

grid = layout([[plot1,plot2]]) 

show(grid)

# Deaths

In [15]:
display(n_death.head(1))

,death_id,province_death_id,case_id,age,sex,health_region,province,country,date_death_report,death_source,additional_info,additional_source
0,1,1,60.0,80-89,Male,Vancouver Coastal,BC,Canada,08-03-2020,https://news.gov.bc.ca/releases/2020HLTH0068-0...,Lynn Valley Resident,NaN


### Setting 

In [16]:
n_death['date_death_report'] = pd.to_datetime(n_death['date_death_report'], dayfirst = True)
n_death['people'] = 1

In [17]:
death_gender = n_death.groupby(['sex'])['people'].sum().sort_values(ascending = False).reset_index();
death_gender_factors = pd.unique(death_gender['sex'])
death_gender_data = ColumnDataSource(death_gender)

death_age = n_death.groupby(['age'])['people'].sum().sort_values(ascending = False).reset_index();
death_age_factors = pd.unique(death_age['age'])
death_age_data = ColumnDataSource(death_age)

death_x = n_death.groupby(['age', 'sex'])['people'].sum().reset_index()
death_x['label'] = tuple(zip(death_x['age'], death_x['sex']))
death_x_data = ColumnDataSource(death_x)

### Deaths By Gender and Age

This chart shows the number of Deaths in gender and age.

In [18]:
plot1 = figure(title = "Deaths By Gender", plot_width = 495, plot_height = 300, x_range = FactorRange(factors = death_gender_factors),background_fill_color="#fafafa")
plot2 = figure(title = "Deaths By Age", plot_width = 495, plot_height = 300, x_range = FactorRange(factors = death_age_factors),background_fill_color="#fafafa")
plot3 = figure(title = "Deaths By Gender & Age", plot_width = 990, plot_height = 300, x_range = FactorRange(factors = death_x['label']),background_fill_color="#fafafa")

plot1.vbar(x = 'sex' , top = 'people' , width = 0.3, source = death_gender_data)
plot2.vbar(x = 'age' , top = 'people' , width = 0.5, source = death_age_data)
plot3.vbar(x = 'label' , top = 'people' , width = 0.8, source = sex_age_data,\
          fill_color=factor_cmap('label', factors= sex_age['sex'] , start=1, end=2, palette = Spectral6))

plot1.add_tools(HoverTool(tooltips = [('Cases', '@people')]))
plot2.add_tools(HoverTool(tooltips = [('Cases', '@people')]))
plot3.add_tools(HoverTool(tooltips = [('Cases', '@people')]))

plot1.xaxis.axis_label = 'Gender'
plot1.yaxis.axis_label = 'Number of Deaths'
plot2.xaxis.axis_label = 'Age'
plot2.yaxis.axis_label = 'Number of Deaths'
plot3.xaxis.axis_label = 'Gender & Age'
plot3.yaxis.axis_label = 'Number of Deaths'

grid = layout([[plot1,plot2],[plot3]]) 

show(grid)

### Setting 

In [19]:
death_report = pd.DataFrame(n_death['date_death_report'].value_counts()).reset_index().sort_values(by = 'index')
death_report_data = ColumnDataSource(death_report)

### Deaths Over Time
This chart shows the number of Deaths on each date.

In [20]:
plot1 = figure(title = 'Deaths Over Time', plot_width = 900, plot_height = 300,x_axis_type="datetime",background_fill_color="#fafafa")
plot1.line(x = 'index' , y = 'date_death_report' , width = 2, source = death_report_data);
plot1.circle(x = 'index', y = 'date_death_report', size= 5, source = death_report_data)
plot1.add_tools(HoverTool(tooltips = [('Cases', '@date_death_report')]))
plot1.xaxis.axis_label = 'Date'
plot1.yaxis.axis_label = 'Number of Deaths'
show(plot1)

### Setting 

In [21]:
province = (pd.DataFrame(n_death['province'].value_counts()).reset_index()
                 .sort_values(by = 'province',ascending = False))
province_factors = pd.unique(province['index'])
province_data = ColumnDataSource(province)

health_region = (pd.DataFrame(n_death['health_region'].value_counts()).reset_index()
                 .sort_values(by = 'health_region',ascending = False))
health_region_factors = pd.unique(health_region['index'])
health_region_data = ColumnDataSource(health_region)

### Deaths By Region
This chart shows the number of deaths in each province and health region.

In [22]:
plot1 = figure(title = "Deaths By Province",plot_width = 990, plot_height = 300,x_range = FactorRange(factors = province_factors),background_fill_color="#fafafa")
plot2 = figure(title = "Deaths By Health Region",plot_width = 990, plot_height = 500,x_range = FactorRange(factors = health_region_factors),background_fill_color="#fafafa")

plot1.vbar(x = 'index' , top = 'province' , width = 0.3, source = province_data);
plot2.vbar(x = 'index' , top = 'health_region' , width = 0.8, source = health_region_data);
plot1.add_tools(HoverTool(tooltips = [('Cases', '@province')]))
plot2.add_tools(HoverTool(tooltips = [('Cases', '@health_region')]))

plot1.xaxis.major_label_orientation = 1
plot2.xaxis.major_label_orientation = 1

plot1.xaxis.axis_label = 'Province'
plot1.yaxis.axis_label = 'Number of Deaths'
plot2.xaxis.axis_label = 'Health Region'
plot2.yaxis.axis_label = 'Number of Deaths'

grid = layout([[plot1],[plot2]]) 

show(grid)

# Testing Cases

In [23]:
display(n_testing.head(1))

,date_testing,province,cumulative_testing
0,29-03-2020,Alberta,44999


### Setting 

In [24]:
n_testing['cumulative_testing'] = n_testing['cumulative_testing'].str.rstrip('*')
n_testing['cumulative_testing'] = n_testing['cumulative_testing'].fillna(0)
n_testing['date_testing'] = pd.to_datetime(n_testing['date_testing'], dayfirst = True)
n_testing = n_testing.sort_values(by = ['province','date_testing']);

In [25]:
test = {}
test_data = {}
for name in n_testing['province'].unique():
    test[name] = pd.DataFrame(n_testing.loc[n_testing['province'] == name])
    test_data[name] = ColumnDataSource(test[name])

### Number of Testing Cases in Canada
This chart shows the number of cumulative testing cases on each date.

In [26]:
plot1 = figure(title = 'Number of Testing Case Over Time', plot_width = 900, plot_height = 900,x_axis_type="datetime",y_range=(0, 60000),background_fill_color="#fafafa")

line1 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Alberta'],color= color[0],legend_label="Alberta");
line2 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['BC'], color= color[1],legend_label="BC");
line3 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Manitoba'], color= color[2],legend_label="Manitoba");
line4 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['NL'], color= color[3],legend_label="NL");
line5 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['NWT'], color= color[4],legend_label="NWT");
line6 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['New Brunswick'], color= color[5],legend_label="New Brunswick");
line7 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nova Scotia'], color= color[6],legend_label="Nova Scotia");
line8 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nunavut'], color= color[7],legend_label="Nunavut");
line9 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Ontario'], color= color[8],legend_label="Ontario");
line10 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['PEI'], color= color[9],legend_label="PEI");
line11 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Quebec'], color= color[10],legend_label="Quebec");
line12 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Saskatchewan'], color= color[11],legend_label="Saskatchewan");
line13 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nova Scotia'], color= color[12],legend_label="Nova Scotia");
line14 = plot1.line(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Yukon'], color= color[13],legend_label="Yukon");

circle1 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Alberta'],color = color[0]);
circle2 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['BC'], color= color[1]);
circle3 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Manitoba'], color= color[2]);
circle4 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['NL'], color= color[3]);
circle5 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['NWT'], color= color[4]);
circle6 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['New Brunswick'], color= color[5]);
circle7 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nova Scotia'], color= color[6]);
circle8 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nunavut'], color= color[7]);
circle9 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Ontario'], color= color[8]);
circle10 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['PEI'], color= color[9]);
circle11 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Quebec'], color= color[10]);
circle12 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Saskatchewan'], color= color[11]);
circle13 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Nova Scotia'], color= color[12]);
circle14 = plot1.circle(x = 'date_testing' , y = 'cumulative_testing' , width = 2, source = test_data['Yukon'], color= color[13]);

line = [line1,line2,line3,line4,line5,line6,line7,line8,line9,line10,line11,line12,line13,line14]

for i in line:
    plot1.add_tools(HoverTool(renderers=[i], tooltips = [('Cases', '@cumulative_testing')]))

plot1.xaxis.axis_label = 'Date'
plot1.yaxis.axis_label = 'Number of Cumulative Testing Case'
plot1.legend.location = 'top_left'
    
grid = layout([[plot1]]) 

show(grid)

# Recoverd Cases

In [27]:
display(n_recovered.head(1))

,date_recovered,province,cumulative_recovered
0,29-03-2020,Alberta,73.0


### Setting 

In [28]:
n_recovered['cumulative_recovered'] = n_recovered['cumulative_recovered'].fillna(0)
n_recovered['date_recovered'] = pd.to_datetime(n_recovered['date_recovered'], dayfirst = True)
n_recovered = n_recovered.sort_values(by = ['province','date_recovered']);

In [29]:
recovered = {}
recovered_data = {}
for name in n_recovered['province'].unique():
    recovered[name] = pd.DataFrame(n_recovered.loc[n_recovered['province'] == name])
    recovered_data[name] = ColumnDataSource(recovered[name])

### Number of Recoverd Cases in Canada
This chart shows the number of cumulative recovered cases on each date.

In [30]:
plot1 = figure(title = 'Number of Recovered Case Over Time', plot_width = 900, plot_height = 600,x_axis_type="datetime",y_range=(1, 420),background_fill_color="#fafafa")

line1 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Alberta'],color = color[0],legend_label="Alberta");
line2 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['BC'], color= color[1],legend_label="BC");
line3 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Manitoba'], color= color[2],legend_label="Manitoba");
line4 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['NL'], color= color[3],legend_label="NL");
line5 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['NWT'], color= color[4],legend_label="NWT");
line6 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['New Brunswick'], color= color[5],legend_label="New Brunswick");
line7 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nova Scotia'], color= color[6],legend_label="Nova Scotia");
line8 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nunavut'], color= color[7],legend_label="Nunavut");
line9 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Ontario'], color= color[8],legend_label="Ontario");
line10 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['PEI'], color= color[9],legend_label="PEI");
line11 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Quebec'], color= color[10],legend_label="Quebec");
line12 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Saskatchewan'], color= color[11],legend_label="Saskatchewan");
line13 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nova Scotia'], color= color[12],legend_label="Nova Scotia");
line14 = plot1.line(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Yukon'], color= color[13],legend_label="Yukon");

circle1 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Alberta'],color = color[0]);
circle2 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['BC'], color= color[1]);
circle3 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Manitoba'], color= color[2]);
circle4 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['NL'], color= color[3]);
circle5 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['NWT'], color= color[4]);
circle6 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['New Brunswick'], color= color[5]);
circle7 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nova Scotia'], color= color[6]);
circle8 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nunavut'], color= color[7]);
circle9 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Ontario'], color= color[8]);
circle10 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['PEI'], color= color[9]);
circle11 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Quebec'], color= color[10]);
circle12 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Saskatchewan'], color= color[11]);
circle13 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Nova Scotia'], color= color[12]);
circle14 = plot1.circle(x = 'date_recovered' , y = 'cumulative_recovered' , width = 2, source = recovered_data['Yukon'], color= color[13]);

line = [line1,line2,line3,line4,line5,line6,line7,line8,line9,line10,line11,line12,line13,line14]

for i in line:
    plot1.add_tools(HoverTool(renderers=[i], tooltips = [('Cases', '@cumulative_recovered')]))

plot1.xaxis.axis_label = 'Date'
plot1.yaxis.axis_label = 'Number of cumulative Recovered Case'
plot1.legend.location = 'top_left'

grid = layout([[plot1]]) 

show(grid)